<a href="https://colab.research.google.com/github/ByeongGil-Jung/2019-Deep-Learning/blob/project_1/project_1/dnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
DNN to classify MNIST handwritten digits
'''

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
"""
Task #1 & Task #2
"""

# Parameters
learning_rate = 0.001
n_epochs = 20
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder(tf.float32, [batch_size, n_input])
Y = tf.placeholder(tf.float32, [batch_size, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(X, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))

#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)


# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training 
    for i in range(n_epochs):
        total_loss = 0.
        n_batches = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for j in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, l = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            # Compute average loss
            total_loss += l
        # Display logs per epoch step
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print("Optimization Finished!")


    correct_preds = tf.equal(tf.argmax(pred, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch   
    
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Average loss epoch 0: 153.05315231323243
Average loss epoch 1: 34.54869186878204
Average loss epoch 2: 21.049974619366907
Average loss epoch 3: 14.835815763370338
Average loss epoch 4: 10.706459432813254
Average loss epoch 5: 8.107694595063792
Average loss epoch 6: 6.216438831114785
Average loss epoch 7: 4.786875377801772
Average loss epoch 8: 3.7467409478243803
Average loss epoch 9: 2.915628507758595
Average loss epoch 10: 2.3468205263353075
Average loss epoch 11: 1.8642536700909957
Average loss epoch 12: 1.4926211005556866
Average loss epoch 13: 1.145184236779962
Average loss epoch 14: 0.9983437227733928
Average loss epoch 15: 0.7785243825667583
Average loss epoch 16: 0.6804330382288742
Average loss epoch 17: 0.54206131573348

In [0]:
"""
Task #3
"""

import pandas as pd
import numpy as np
import time

task3_data_dict = {"Features": np.array([256, 128, 64]),
                   "Accuracy": np.zeros(3),
                   "Num of Hidden Layers": np.zeros(3),
                   "Num of Learning Params": np.zeros(3),
                   "Avg Loss (Last Epoch)": np.zeros(3),
                   "Elapsed Time": np.zeros(3)}

task3_df = pd.DataFrame.from_dict(task3_data_dict)
task3_df.index = task3_df["Features"]
del task3_df["Features"]

In [4]:
"""
Task #3 - 1 : DNN with two hidden layers, each layer has 256 units (as in the given code)
"""

# Pandas Params
row_index = 256

# Parameters
learning_rate = 0.001
n_epochs = 20
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

task3_df.loc[row_index, "Num of Hidden Layers"] = 2
task3_df.loc[row_index, "Num of Learning Params"] = n_input * n_hidden_1 * n_hidden_2

# tf Graph input
X = tf.placeholder(tf.float32, [batch_size, n_input])
Y = tf.placeholder(tf.float32, [batch_size, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(X, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))

#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)


# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    start = time.time()
    # Training 
    for i in range(n_epochs):
        total_loss = 0.
        n_batches = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for j in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, l = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            # Compute average loss
            total_loss += l
        # Display logs per epoch step
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        
        if (j == (n_batches - 1)):
            task3_df.loc[row_index, "Avg Loss (Last Epoch)"] = total_loss / n_batches
            

    end = time.time()
    print("Optimization Finished!")

    task3_df.loc[row_index, "Elapsed Time"] = end - start

    correct_preds = tf.equal(tf.argmax(pred, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch   
    
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))
    
    task3_df.loc[row_index, "Accuracy"] = total_correct_preds / mnist.test.num_examples

Average loss epoch 0: 161.75211776733397
Average loss epoch 1: 32.77292487231168
Average loss epoch 2: 19.97018756527792
Average loss epoch 3: 13.901552612944084
Average loss epoch 4: 10.347553300139579
Average loss epoch 5: 7.60185929791242
Average loss epoch 6: 5.813232704766897
Average loss epoch 7: 4.433505963434676
Average loss epoch 8: 3.499914564762364
Average loss epoch 9: 2.7621702762940132
Average loss epoch 10: 2.135802597118354
Average loss epoch 11: 1.6886929274170297
Average loss epoch 12: 1.3922709836354998
Average loss epoch 13: 1.0771479788235583
Average loss epoch 14: 0.9805187265026399
Average loss epoch 15: 0.7792720241309984
Average loss epoch 16: 0.620552207996282
Average loss epoch 17: 0.5524935171091884
Average loss epoch 18: 0.4588337336576273
Average loss epoch 19: 0.3893880875661087
Optimization Finished!
Accuracy 0.948


In [5]:
"""
Task #3 - 2 : DNN with three hidden layers, each layer has 128 units
"""

# Pandas Params
row_index = 128

# Parameters
learning_rate = 0.001
n_epochs = 20
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 128 # 1st layer number of features
n_hidden_2 = 128 # 2nd layer number of features
n_hidden_3 = 128 # 3rd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

task3_df.loc[row_index, "Num of Hidden Layers"] = 3
task3_df.loc[row_index, "Num of Learning Params"] = n_input * n_hidden_1 * n_hidden_2 * n_hidden_3

# tf Graph input
X = tf.placeholder(tf.float32, [batch_size, n_input])
Y = tf.placeholder(tf.float32, [batch_size, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Hidden layer
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(X, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))

#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)


# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    start = time.time()
    # Training 
    for i in range(n_epochs):
        total_loss = 0.
        n_batches = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for j in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, l = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            # Compute average loss
            total_loss += l
        # Display logs per epoch step
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        
        if (j == (n_batches - 1)):
            task3_df.loc[row_index, "Avg Loss (Last Epoch)"] = total_loss / n_batches
            

    end = time.time()
    print("Optimization Finished!")

    task3_df.loc[row_index, "Elapsed Time"] = end - start

    correct_preds = tf.equal(tf.argmax(pred, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch   
    
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))
    
    task3_df.loc[row_index, "Accuracy"] = total_correct_preds / mnist.test.num_examples

Average loss epoch 0: 605.0163947642933
Average loss epoch 1: 119.64037280342795
Average loss epoch 2: 71.04745685490695
Average loss epoch 3: 49.49430141860788
Average loss epoch 4: 36.88555232893337
Average loss epoch 5: 28.535161086971108
Average loss epoch 6: 22.69612650979649
Average loss epoch 7: 18.15500311356553
Average loss epoch 8: 14.703215063823896
Average loss epoch 9: 12.063685850633348
Average loss epoch 10: 10.234946236584047
Average loss epoch 11: 8.492248942470095
Average loss epoch 12: 7.089848179589
Average loss epoch 13: 5.984559438397651
Average loss epoch 14: 5.095772584290289
Average loss epoch 15: 4.432371449888431
Average loss epoch 16: 3.8694188242309777
Average loss epoch 17: 3.2842042136855194
Average loss epoch 18: 2.8399111003662147
Average loss epoch 19: 2.3456248241145605
Optimization Finished!
Accuracy 0.9331


In [6]:
"""
Task #3 - 3 : DNN with four hidden layers, each layer has 64 units
"""

# Pandas Params
row_index = 64

# Parameters
learning_rate = 0.001
n_epochs = 20
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 64 # 1st layer number of features
n_hidden_2 = 64 # 2nd layer number of features
n_hidden_3 = 64 # 3rd layer number of features
n_hidden_4 = 64 # 4th layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

task3_df.loc[row_index, "Num of Hidden Layers"] = 4
task3_df.loc[row_index, "Num of Learning Params"] = n_input * n_hidden_1 * n_hidden_2 * n_hidden_3 * n_hidden_4

# tf Graph input
X = tf.placeholder(tf.float32, [batch_size, n_input])
Y = tf.placeholder(tf.float32, [batch_size, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Hidden layer
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    # Hidden layer
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(X, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))

#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)


# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    start = time.time()
    # Training 
    for i in range(n_epochs):
        total_loss = 0.
        n_batches = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for j in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, l = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            # Compute average loss
            total_loss += l
        # Display logs per epoch step
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        
        if (j == (n_batches - 1)):
            task3_df.loc[row_index, "Avg Loss (Last Epoch)"] = total_loss / n_batches
            

    end = time.time()
    print("Optimization Finished!")

    task3_df.loc[row_index, "Elapsed Time"] = end - start

    correct_preds = tf.equal(tf.argmax(pred, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch   
    
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))
    
    task3_df.loc[row_index, "Accuracy"] = total_correct_preds / mnist.test.num_examples

Average loss epoch 0: 1240.6651515059039
Average loss epoch 1: 218.95880047191272
Average loss epoch 2: 124.72761780478737
Average loss epoch 3: 84.02891473770141
Average loss epoch 4: 62.18773619044911
Average loss epoch 5: 48.38673423420299
Average loss epoch 6: 38.263797568407924
Average loss epoch 7: 31.384788921529598
Average loss epoch 8: 26.305756939866328
Average loss epoch 9: 22.26959834142165
Average loss epoch 10: 19.12652452924035
Average loss epoch 11: 16.69009042208845
Average loss epoch 12: 14.66473642912778
Average loss epoch 13: 12.956089394526048
Average loss epoch 14: 11.60253980926492
Average loss epoch 15: 10.41098059696738
Average loss epoch 16: 9.322097208946943
Average loss epoch 17: 8.360155446509069
Average loss epoch 18: 7.6526207690618255
Average loss epoch 19: 7.000530377075347
Optimization Finished!
Accuracy 0.9054


In [7]:
task3_df

,Accuracy,Avg Loss (Last Epoch),Elapsed Time,Num of Hidden Layers,Num of Learning Params
Features,,,,,
256,0.9480,0.389388,27.174656,2.0,5.138022e+07
128,0.9331,2.345625,28.431317,3.0,1.644167e+09
64,0.9054,7.000530,30.036170,4.0,1.315334e+10


In [8]:
"""
Feature 의 갯수가 적고, Layer 가 깊어질 수록 
"""

'\nFeature 의 갯수가 적고, Layer 가 깊어질 수록 \n'

In [0]:
"""
Task #4
"""
model_data_dict = dict()

# Network Parameters
n_hidden_1 = 512 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# 784 -> 512 -> 256 -> 10
class MyModel_1(object):
    
    def __init__(self, sess, learning_rate, batch_size):
        self.sess = sess
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.hidden_1_size = 512
        self.hidden_2_size = 256
        
        self.X = tf.placeholder(tf.float32, shape=[self.batch_size, n_input])
        self.Y = tf.placeholder(tf.float32, shape=[self.batch_size, n_classes])
        
        self.pred = None
        self.loss = None
        self.optimizer = None
        
        self.correct_preds = None
        self.accuracy = None
        
    def build_model(self):
        with tf.variable_scope("Layer1"):
            W1 = tf.Variable(tf.random_normal([n_input, self.hidden_1_size]))
            b1 = tf.Variable(tf.random_normal([self.hidden_1_size]))
            layer1 = tf.add(tf.matmul(self.X, W1), b1)
            layer1 = tf.nn.relu(layer1)
            
        with tf.variable_scope("Layer2"):
            W2 = tf.Variable(tf.random_normal([self.hidden_1_size, self.hidden_2_size]))
            b2 = tf.Variable(tf.random_normal([self.hidden_2_size]))
            layer2 = tf.add(tf.matmul(layer1, W2), b2)
            layer2 = tf.nn.relu(layer2)
        
        with tf.variable_scope("Layer3"):
            W3 = tf.Variable(tf.random_normal([self.hidden_2_size, n_classes]))
            b3 = tf.Variable(tf.random_normal([n_classes]))
            layer3 = tf.add(tf.matmul(layer2, W3), b3)
        
        self.pred = layer3
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.pred, labels=self.Y))
        self.optimizer = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        
        self.correct_preds = tf.equal(tf.argmax(self.pred, axis=1), tf.argmax(self.Y, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_preds, tf.float32))
    
    def train(self, x_batch, y_batch):
        return self.sess.run(fetches=[self.loss, self.optimizer], feed_dict={self.X: x_batch, self.Y: y_batch})
    
    def get_accuracy(self, x_batch, y_batch):
        return self.sess.run(fetches=[self.accuracy], feed_dict={self.X: x_batch, self.Y: y_batch})


# 784 -> 784 -> 512 -> 10
class MyModel_2(object):
    
    def __init__(self, sess, learning_rate, batch_size):
        self.sess = sess
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.hidden_1_size = 784
        self.hidden_2_size = 512
        
        self.X = tf.placeholder(tf.float32, shape=[self.batch_size, n_input])
        self.Y = tf.placeholder(tf.float32, shape=[self.batch_size, n_classes])
        
        self.pred = None
        self.loss = None
        self.optimizer = None
        
        self.correct_preds = None
        self.accuracy = None
        
    def build_model(self):
        with tf.variable_scope("Layer1"):
            W1 = tf.Variable(tf.random_normal([n_input, self.hidden_1_size]))
            b1 = tf.Variable(tf.random_normal([self.hidden_1_size]))
            layer1 = tf.add(tf.matmul(self.X, W1), b1)
            layer1 = tf.nn.relu(layer1)
            
        with tf.variable_scope("Layer2"):
            W2 = tf.Variable(tf.random_normal([self.hidden_1_size, self.hidden_2_size]))
            b2 = tf.Variable(tf.random_normal([self.hidden_2_size]))
            layer2 = tf.add(tf.matmul(layer1, W2), b2)
            layer2 = tf.nn.relu(layer2)
        
        with tf.variable_scope("Layer3"):
            W3 = tf.Variable(tf.random_normal([self.hidden_2_size, n_classes]))
            b3 = tf.Variable(tf.random_normal([n_classes]))
            layer3 = tf.add(tf.matmul(layer2, W3), b3)
        
        self.pred = layer3
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.pred, labels=self.Y))
        self.optimizer = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        
        self.correct_preds = tf.equal(tf.argmax(self.pred, axis=1), tf.argmax(self.Y, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_preds, tf.float32))
        
    def train(self, x_batch, y_batch):
        return self.sess.run(fetches=[self.loss, self.optimizer], feed_dict={self.X: x_batch, self.Y: y_batch})
    
    def get_accuracy(self, x_batch, y_batch):
        return self.sess.run(fetches=[self.accuracy], feed_dict={self.X: x_batch, self.Y: y_batch})

In [10]:
"""
Model 1 (MyModel_1)
"""

# Parameters
n_epochs_list = [10, 20, 40, 80, 160, 320]
batch_size_list = [100, 200]
learning_rate_list = [1e-1, 1e-2, 1e-3]
display_step = 1

# n_epochs = 10
# batch_size = 100
# learning_rate = 1e-3


# Launch the graph
model_1_dict = dict()

with tf.Session() as sess:
    #Training
    n_epochs_dict = dict()
    for n_epochs in n_epochs_list:
        batch_size_dict = dict()
        
        for batch_size in batch_size_list:
            learning_rate_dict = dict()
            
            for learning_rate in learning_rate_list:
                accuracy_dict = dict()
                
                model = MyModel_1(sess=sess, learning_rate=learning_rate, batch_size=batch_size)
                model.build_model()
                
                # Start
                sess.run(tf.global_variables_initializer())
                
                print("n_epochs : {}, batch_size : {}, learning_rate : {}".format(n_epochs, batch_size, learning_rate))
                
                for i in range(n_epochs):
                    total_loss = 0.
                    n_batches = int(mnist.train.num_examples/batch_size)
                    # Loop over all batches
                    for j in range(n_batches):
                        X_batch, Y_batch = mnist.train.next_batch(batch_size)
                        # Run optimization op (backprop) and cost op (to get loss value)
                        l, _ = model.train(x_batch=X_batch, y_batch=Y_batch)
                        # Compute average loss
                        total_loss += l
                    # Display logs per epoch step
                    # print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

                # print("Optimization Finished!")

                n_batches = int(mnist.test.num_examples/batch_size)
                total_correct_preds = 0

                for i in range(n_batches):
                    X_batch, Y_batch = mnist.test.next_batch(batch_size)
                    accuracy_batch = model.get_accuracy(x_batch=X_batch, y_batch=Y_batch)
                    total_correct_preds += accuracy_batch[0]

                accuracy = total_correct_preds / n_batches
                print('> Accuracy : {0}'.format(accuracy))
                
                learning_rate_dict[learning_rate] = accuracy
            
            batch_size_dict[batch_size] = learning_rate_dict
            
        n_epochs_dict[n_epochs] = batch_size_dict
    
    model_1_dict["model_1"] = n_epochs_dict

n_epochs : 10, batch_size : 100, learning_rate : 0.1


KeyboardInterrupt: ignored

In [73]:
model_1_dict

{'model_1': {10: {100: {0.001: 0.9501999980211258}}}}